# sbert

In [29]:
import importlib
import data_preprocessing  # Make sure it's already imported
importlib.reload(data_preprocessing)  # Force reload

<module 'data_preprocessing' from '/home/onyxia/work/data_preprocessing.py'>

In [30]:
import pandas as pd
import numpy as np
import data_preprocessing
from data_preprocessing import clean_text, correct_spelling, replace_emoji

In [18]:
df = pd.read_parquet("dataset.parquet")
label_df = df[df['label'] == True]

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_train_label, df_test_label = train_test_split(label_df, test_size=0.2,random_state=42)

In [31]:
# data preprocessing, skip lemmanization for bert and sbert
def process_text_pipeline(text, country):
    text = clean_text(text)
    text = correct_spelling(text, country)
    text = replace_emoji(text, country)
    return text

# process for df_train
processed_texts = []
# Iterate through the DataFrame rows
for index, row in df_train_label.iterrows():
    processed_text = process_text_pipeline(row['quote_text'], row['country_name'])
    processed_texts.append(processed_text)

# Assign lists back to the DataFrame
df_train_label['processed_text_nsc'] = processed_texts

# Repeat for df_test
processed_texts = []

for index, row in df_test_label.iterrows():
    processed_text = process_text_pipeline(row['quote_text'], row['country_name'])
    processed_texts.append(processed_text)

df_test_label['processed_text_nsc'] = processed_texts


In [ ]:
# Sentence-BERT (SBERT)
from sentence_transformers import SentenceTransformer
# Load pre-trained SBERT model
sbert_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

def get_sbert_embedding(text):
    """Generates sentence embedding using SBERT."""
    return sbert_model.encode(text)

# get embedding for training set
embedded_texts = []

# Iterate through the DataFrame rows
for index, row in df_train_label.iterrows():
    embedded_text = get_sbert_embedding(row['processed_text_nsc'])
    embedded_texts.append(embedded_text)

# Assign lists back to the DataFrame
df_train_label['sbert_embedding'] = embedded_texts

# for test set
embedded_texts = []

# Iterate through the DataFrame rows
for index, row in df_test_label.iterrows():
    embedded_text = get_sbert_embedding(row['processed_text_nsc'])
    embedded_texts.append(embedded_text)

# Assign lists back to the DataFrame
df_test_label['sbert_embedding'] = embedded_texts


In [68]:
# Assign lists back to the DataFrame
df_train_label['sbert_embedding'] = embedded_texts

# for test set
embedded_texts = []

# Iterate through the DataFrame rows
for index, row in df_test_label.iterrows():
    embedded_text = get_sbert_embedding(row['processed_text_nsc'])
    embedded_texts.append(embedded_text)

# Assign lists back to the DataFrame
df_test_label['sbert_embedding'] = embedded_texts

In [67]:
len(embedded_text)

384

In [69]:
from sklearn.cluster import KMeans
import numpy as np

# Prepare SBERT embeddings
# Convert embeddings from lists to NumPy arrays
df_train_label['sbert_embedding'] = df_train_label['sbert_embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else np.array(x))
# Stack embeddings properly
X = np.vstack(df_train_label['sbert_embedding'].values)

# Apply K-Means clustering
k = 11  # Number of clusters, 10 topics add out of topic
kmeans = KMeans(n_clusters=k, random_state=42)
df_train_label['cluster'] = kmeans.fit_predict(X)

In [71]:
# Group by cluster and topic_id to see distribution
cluster_topic_distribution = df_train_label.groupby(['cluster', 'topic_id']).size().reset_index(name='count')

# Find the most common topic_id in each cluster
cluster_to_topic = cluster_topic_distribution.sort_values('count', ascending=False).drop_duplicates('cluster')

# Create a mapping from cluster to topic_id
cluster_topic_mapping = dict(zip(cluster_to_topic['cluster'], cluster_to_topic['topic_id']))

# Assign the matched topic_id back to the DataFrame
df_train_label['matched_topic_id'] = df_train_label['cluster'].map(cluster_topic_mapping)

# Check the mapping
print(df_train_label[['quote_text', 'cluster', 'topic_id', 'matched_topic_id']].head())

                                              quote_text  cluster  topic_id  \
38030  The face cream was easy to pump out and I didn...        5     550.0   
50679  Très bonne idée la recharge, pour partir en vo...        2     556.0   
35584  J’utilise ce produit depuis 2 mois maintenant ...        5     546.0   
44679  This set of 3 masks comes in a fully recycled ...       10     602.0   
33645  Points forts    Format automatique pratique, s...        2     546.0   

       matched_topic_id  
38030             550.0  
50679             546.0  
35584             550.0  
44679             602.0  
33645             546.0  


In [74]:
# predict for df_test
df_test_label['sbert_embedding'] = df_test_label['sbert_embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else np.array(x))
X = np.vstack(df_test_label['sbert_embedding'].values)#.astype(np.float64)
df_test_label['cluster'] = kmeans.predict(X)
# map to the topic id
df_test_label['matched_topic_id'] = df_test_label['cluster'].map(cluster_topic_mapping)

In [75]:
train_accuracy = np.sum(df_train_label['matched_topic_id'] == df_train_label['topic_id'])/len(df_train_label)
test_accuracy = np.sum(df_test_label['matched_topic_id'] == df_test_label['topic_id'])/len(df_test_label)
print('train accuracy:', train_accuracy)
print('test accuracy:', test_accuracy)

train accuracy: 0.4351297405189621
test accuracy: 0.4421838784445017


# bert

In [ ]:
!pip install transformers torch

In [41]:
from transformers import BertTokenizer, BertModel
import torch

# Load BERT base model and tokenizer
# for now try the most basic pretrained
bert_model_name = "bert-base-uncased"  
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)


In [43]:
def get_bert_word_embeddings(text, model, tokenizer):
    """
    Extracts word embeddings from BERT for a given text.

    Parameters:
    - text (str): Input text
    - model (transformers.BertModel): Pretrained BERT model
    - tokenizer (transformers.BertTokenizer): BERT tokenizer

    Returns:
    - word_embeddings (numpy array): Word-level embeddings
    """
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)

    # Get last hidden states (shape: [batch_size, num_tokens, embedding_size])
    last_hidden_state = outputs.last_hidden_state
    return last_hidden_state.squeeze(0).numpy()  # Convert tensor to numpy



In [ ]:
embedded_texts = []

# Iterate through the DataFrame rows
for index, row in df_train_label.iterrows():
    embedded_text = get_bert_word_embeddings(row['processed_text_nsc'], bert_model, tokenizer)
    embedded_texts.append(embedded_text)

# Assign lists back to the DataFrame
df_train_label['bert_embedding'] = embedded_texts


In [51]:
# for test set
embedded_texts = []

# Iterate through the DataFrame rows
for index, row in df_test_label.iterrows():
    embedded_text = get_bert_word_embeddings(row['processed_text_nsc'], bert_model, tokenizer)
    embedded_texts.append(embedded_text)

# Assign lists back to the DataFrame
df_test_label['bert_embedding'] = embedded_texts

In [57]:
import numpy as np

def mean_pooling(embedding):
    """ Convert variable-length BERT word embeddings to a fixed-size 768D vector (sentence embedding). """
    return np.mean(embedding, axis=0) if embedding.shape[0] > 1 else embedding.squeeze()

# Apply mean pooling to all embeddings
df_train_label["bert_embedding"] = df_train_label["bert_embedding"].apply(mean_pooling)
df_test_label["bert_embedding"] = df_test_label["bert_embedding"].apply(mean_pooling)


In [58]:
X_train = np.vstack(df_train_label["bert_embedding"].values)  # Now shape: (num_samples, 768)
X_test = np.vstack(df_test_label["bert_embedding"].values)


In [59]:
from sklearn.cluster import KMeans

num_clusters = 11  # Adjust based on your dataset
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)

# Fit K-Means to sentence-level embeddings
kmeans.fit(X_train)

# Assign clusters
df_train_label["cluster"] = kmeans.labels_
df_test_label["cluster"] = kmeans.predict(X_test)

print(df_train_label["cluster"].value_counts())  # Check cluster distribution


cluster
0     2336
2     2055
6     1948
8     1628
10    1543
3     1423
1     1334
9      990
7      979
5      849
4      446
Name: count, dtype: int64


In [61]:
# Group by cluster and topic_id to see distribution
cluster_topic_distribution = df_train_label.groupby(['cluster', 'topic_id']).size().reset_index(name='count')

# Find the most common topic_id in each cluster
cluster_to_topic = cluster_topic_distribution.sort_values('count', ascending=False).drop_duplicates('cluster')

# Create a mapping from cluster to topic_id
cluster_topic_mapping = dict(zip(cluster_to_topic['cluster'], cluster_to_topic['topic_id']))

# Assign the matched topic_id back to the DataFrame
df_train_label['matched_topic_id'] = df_train_label['cluster'].map(cluster_topic_mapping)
df_test_label['matched_topic_id'] = df_test_label['cluster'].map(cluster_topic_mapping)

In [62]:
train_accuracy = np.sum(df_train_label['matched_topic_id'] == df_train_label['topic_id'])/len(df_train_label)
test_accuracy = np.sum(df_test_label['matched_topic_id'] == df_test_label['topic_id'])/len(df_test_label)
print('train accuracy:', train_accuracy)
print('test accuracy:', test_accuracy)

train accuracy: 0.2832399716695641
test accuracy: 0.2719546742209632
